# Recovery Data Partnership - SafeGraph user guide

### 1. Install Dependencies
uncomment the line below and run it

In [7]:
# ! pip3 install s3fs pandas

In [1]:
import pandas as pd
import s3fs

We will be using `s3fs` to access our files in aws s3. Authentication is handled as below:

In [5]:
s3 = s3fs.S3FileSystem(
      key='<YOUR KEY HERE>',
      secret='<YOUR SECRET HERE>', 
      client_kwargs={
          'endpoint_url': 'https://s3.amazonaws.com', 
          'region_name':'us-east-1'
      }
    )

### 2. Listing available files

`s3.ls` will allow you to list directories given s3 path. For this project specifically, we will use `recovery-data-partnership/output` folder as the root folder for delivering all output files


In [3]:
s3.ls('recovery-data-partnership/output/')

['recovery-data-partnership/output/example',
 'recovery-data-partnership/output/lookups',
 'recovery-data-partnership/output/poi',
 'recovery-data-partnership/output/social_distancing']

Under the `output` folder, you will find data products by category. All social distancing related data products will be stored under `social_distancing`, and all point of interest and patterns datasets will be stored under `poi`. 

In [4]:
s3.ls('recovery-data-partnership/output/social_distancing/')

['recovery-data-partnership/output/social_distancing/device_counts_by_county.csv.zip',
 'recovery-data-partnership/output/social_distancing/device_counts_by_state.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_county_trips',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips']

Because of the large size of the datasets, we paritioned each output table by year and quarter.

In [5]:
s3.ls('recovery-data-partnership/output/social_distancing/weekly_state_trips/')

['recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q1.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q2.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q3.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q4.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q1.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q2.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q3.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q4.csv.zip']

### 3. Reading files to dataframe and combine them together

You can easily write a loop to loop through all the year quarter partitions of the datasets and use `pd.concat` to concatenate paritions into one big table.

In [6]:
dfs = []
for dataset in s3.ls('recovery-data-partnership/output/social_distancing/weekly_state_trips/'):
  df = pd.read_csv(s3.open(dataset, mode='rb'), compression='zip')
  dfs.append(df)
  del df

In [7]:
weekly_state_trips = pd.concat(dfs)

> Note that because we are paritioning by quarter, that means the last week of a quarter can cross quarters and show up in two paritions. To avoid confusion, we recommend doing a group by by week and the geographic boundry.

In [11]:
weekly_state_trips = weekly_state_trips.groupby(['year_week', 'origin', 'destination']).sum().reset_index()

In [12]:
weekly_state_trips

,year_week,origin,destination,weekday_trips,weekend_trips,all_trips
0,2019-01,01,NYC,3167.0,493.0,3660
1,2019-01,02,NYC,160.0,13.0,173
2,2019-01,04,NYC,2852.0,362.0,3214
3,2019-01,05,NYC,1308.0,274.0,1582
4,2019-01,06,NYC,24119.0,3416.0,27535
...,...,...,...,...,...,...
10166,2020-42,NYC,55,321.0,0.0,321
10167,2020-42,NYC,56,97.0,0.0,97
10168,2020-42,NYC,72,429.0,0.0,429
10169,2020-42,NYC,78,12.0,0.0,12


### 4. We also prepared look up tables

In [13]:
s3.ls('recovery-data-partnership/output/lookups/')

['recovery-data-partnership/output/lookups/',
 'recovery-data-partnership/output/lookups/fips_to_county.csv',
 'recovery-data-partnership/output/lookups/fips_to_state.csv',
 'recovery-data-partnership/output/lookups/naics_sector.csv',
 'recovery-data-partnership/output/lookups/naics_subsector.csv',
 'recovery-data-partnership/output/lookups/nta_to_boro_county.csv']

The `state_lookup` table will give you the state names

In [20]:
state_lookup = pd.read_csv(s3.open('recovery-data-partnership/output/lookups/fips_to_state.csv', mode='rb'), dtype=str)
state_lookup.head()

,fips,state
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [21]:
named_weekly_state_trips = pd.merge(weekly_state_trips, state_lookup, how='left', left_on='origin', right_on='fips')
named_weekly_state_trips.head()

,year_week,origin,destination,weekday_trips,weekend_trips,all_trips,fips,state
0,2019-01,01,NYC,3167.0,493.0,3660,01,Alabama
1,2019-01,02,NYC,160.0,13.0,173,02,Alaska
2,2019-01,04,NYC,2852.0,362.0,3214,04,Arizona
3,2019-01,05,NYC,1308.0,274.0,1582,05,Arkansas
4,2019-01,06,NYC,24119.0,3416.0,27535,06,California


The device count table for states contains the number of devices that's "home" is in each state. Because the state trips table concerns movement in and out of NYC, the device counts for NY do not include the 5 boroughs. These counts are instead stored as state = 'NYC'. Also note that the devices counts are only associated with the origin location. 


In [23]:
state_devices = pd.read_csv(s3.open('recovery-data-partnership/output/social_distancing/device_counts_by_state.csv.zip', mode='rb'),  compression='zip')
state_devices.head()

,year_week,state,device_count,completely_home_device_count
0,2019-15,NYC,511057,180419
1,2019-44,NYC,398584,110357
2,2020-04,NYC,336748,84469
3,2019-23,NYC,514242,158430
4,2020-01,10,55314,19195


In [25]:
weekly_state_trips_with_devices = pd.merge(
    weekly_state_trips, state_devices, 
    how='left', 
    left_on=['origin', 'year_week'],
    right_on=['state', 'year_week'])

In [26]:
weekly_state_trips_with_devices.head()

,year_week,origin,destination,weekday_trips,weekend_trips,all_trips,state,device_count,completely_home_device_count
0,2019-01,01,NYC,3167.0,493.0,3660,01,57,24
1,2019-01,01,NYC,3167.0,493.0,3660,01,412283,119087
2,2019-01,01,NYC,3167.0,493.0,3660,01,59,15
3,2019-01,01,NYC,3167.0,493.0,3660,01,57,27
4,2019-01,01,NYC,3167.0,493.0,3660,01,410832,138154
